## 5 folds cross-validation: 60/% train - 20/% validation - 20%test

In [1]:
import os
import numpy as np
import shutil
from sklearn.model_selection import StratifiedKFold, train_test_split

def prepare_and_save_folds(train_dir, test_dir, folds_dir, id, n_splits_outer=5):

    if os.path.exists(folds_dir):
        shutil.rmtree(folds_dir)
        print(f"Carpeta '{folds_dir}' eliminada.")
    os.makedirs(folds_dir)
    print(f"Carpeta '{folds_dir}' creada.")

    train_files = [f for f in os.listdir(train_dir) if f.endswith('_X.npy')]
    labels = [0 if 'AVPEPUDEAC' in f else 1 for f in train_files]

    test_files = [f for f in os.listdir(test_dir) if f.endswith('_X.npy')]
    test_labels = [0 if 'AVPEPUDEAC' in f else 1 for f in test_files]
    print(train_files==test_files)

    skf_outer = StratifiedKFold(n_splits=n_splits_outer, shuffle=True, random_state=42)

    for fold_idx, (train_val_index, test_index) in enumerate(skf_outer.split(train_files, labels)):
        # Print the test files for the current fold
        print(f"Fold {fold_idx + 1} Test Files:")
        for i in test_index:
            print(f"File: {test_files[i]}, Label: {test_labels[i]}")

        # Load test data
        test_data = [np.load(os.path.join(test_dir, test_files[i])) for i in test_index]
        test_data = np.vstack(test_data)
        test_labels_fold = np.array([test_labels[i] for i in test_index])
        test_labels_fold = np.repeat(test_labels_fold, test_data.shape[0] // len(test_labels_fold))
        test_data, test_labels_fold = shuffle_data(test_data, test_labels_fold)  # Shuffle test data


        # Assign data to train and validation using the inner split
        train_index, val_index = train_test_split(train_val_index, test_size=20, stratify=[labels[i] for i in train_val_index], shuffle=True, random_state=42)

        # Print the train files for the current fold
        print(f"Fold {fold_idx + 1} Train Files:")
        for i in train_index:
            print(f"File: {train_files[i]}, Label: {labels[i]}")

        # Print the validation files for the current fold
        print(f"Fold {fold_idx + 1} Validation Files:")
        for i in val_index:
            print(f"File: {train_files[i]}, Label: {labels[i]}")

        train_data = [np.load(os.path.join(train_dir, train_files[i])) for i in train_index]
        train_data = np.vstack(train_data)
        train_labels = np.array([labels[i] for i in train_index])
        train_labels = np.repeat(train_labels, train_data.shape[0] // len(train_labels))
        train_data, train_labels = shuffle_data(train_data, train_labels)  # Shuffle train data

        val_data = [np.load(os.path.join(train_dir, train_files[i])) for i in val_index]
        val_data = np.vstack(val_data)
        val_labels = np.array([labels[i] for i in val_index])
        val_labels = np.repeat(val_labels, val_data.shape[0] // len(val_labels))
        val_data, val_labels = shuffle_data(val_data, val_labels)


        # Save the data to .npy files
        np.save(os.path.join(folds_dir, f'X_train_{id}_fold_{fold_idx+1}.npy'), train_data)
        np.save(os.path.join(folds_dir, f'y_train_{id}_fold_{fold_idx+1}.npy'), train_labels)
        np.save(os.path.join(folds_dir, f'X_val_{id}_fold_{fold_idx+1}.npy'), val_data)
        np.save(os.path.join(folds_dir, f'y_val_{id}_fold_{fold_idx+1}.npy'), val_labels)
        np.save(os.path.join(folds_dir, f'X_test_{id}_fold_{fold_idx+1}.npy'), test_data)
        np.save(os.path.join(folds_dir, f'y_test_{id}_fold_{fold_idx+1}.npy'), test_labels_fold)

        print(train_index, val_index, test_index)

    print("Archivos .npy guardados para cada fold en la carpeta de folds.")

def shuffle_data(data, labels):
    shuffle_indices = np.random.permutation(len(labels))
    data = data[shuffle_indices]
    labels = labels[shuffle_indices]
    return data, labels

In [2]:
def prepare_and_save_folds_mixed(train_dir_real, train_dir_syn, test_dir_real, test_dir_syn, folds_dir, id, n_splits_outer=5):
    """
    Prepara y guarda folds mezclando datos reales y sintéticos tanto para entrenamiento como para evaluación.
    
    Args:
        train_dir_real: Directorio con archivos de entrenamiento reales
        train_dir_syn: Directorio con archivos de entrenamiento sintéticos
        test_dir_real: Directorio con archivos de test reales
        test_dir_syn: Directorio con archivos de test sintéticos
        folds_dir: Directorio donde se guardarán los folds
        id: Identificador para los archivos
        n_splits_outer: Número de folds (default: 5)
    """
    
    if os.path.exists(folds_dir):
        shutil.rmtree(folds_dir)
        print(f"Carpeta '{folds_dir}' eliminada.")
    os.makedirs(folds_dir)
    print(f"Carpeta '{folds_dir}' creada.")

    # Cargar archivos de entrenamiento (reales + sintéticos)
    train_files_real = [f for f in os.listdir(train_dir_real) if f.endswith('_X.npy')]
    train_files_syn = [f for f in os.listdir(train_dir_syn) if f.endswith('_X.npy')]
    
    # Combinar archivos de entrenamiento
    train_files = train_files_real + train_files_syn
    train_dirs = [train_dir_real] * len(train_files_real) + [train_dir_syn] * len(train_files_syn)
    labels = [0 if 'AVPEPUDEAC' in f else 1 for f in train_files]
    
    # Cargar archivos de test (reales + sintéticos)
    test_files_real = [f for f in os.listdir(test_dir_real) if f.endswith('_X.npy')]
    test_files_syn = [f for f in os.listdir(test_dir_syn) if f.endswith('_X.npy')]
    
    # Combinar archivos de test
    test_files = test_files_real + test_files_syn
    test_dirs = [test_dir_real] * len(test_files_real) + [test_dir_syn] * len(test_files_syn)
    test_labels = [0 if 'AVPEPUDEAC' in f else 1 for f in test_files]
    
    print(f"Total archivos entrenamiento: {len(train_files)} (Real: {len(train_files_real)}, Sintético: {len(train_files_syn)})")
    print(f"Total archivos test: {len(test_files)} (Real: {len(test_files_real)}, Sintético: {len(test_files_syn)})")

    skf_outer = StratifiedKFold(n_splits=n_splits_outer, shuffle=True, random_state=42)

    for fold_idx, (train_val_index, test_index) in enumerate(skf_outer.split(train_files, labels)):
        # Print the test files for the current fold
        print(f"\nFold {fold_idx + 1} Test Files:")
        for i in test_index:
            print(f"File: {test_files[i]}, Label: {test_labels[i]}")

        # Load test data from both real and synthetic sources
        test_data = []
        for i in test_index:
            data = np.load(os.path.join(test_dirs[i], test_files[i]))
            test_data.append(data)
        test_data = np.vstack(test_data)
        test_labels_fold = np.array([test_labels[i] for i in test_index])
        test_labels_fold = np.repeat(test_labels_fold, test_data.shape[0] // len(test_labels_fold))
        test_data, test_labels_fold = shuffle_data(test_data, test_labels_fold)  # Shuffle test data

        # Assign data to train and validation using the inner split
        train_index, val_index = train_test_split(
            train_val_index, 
            test_size=20, 
            stratify=[labels[i] for i in train_val_index], 
            shuffle=True, 
            random_state=42
        )

        # Print the train files for the current fold
        print(f"Fold {fold_idx + 1} Train Files:")
        for i in train_index:
            print(f"File: {train_files[i]}, Label: {labels[i]}")

        # Print the validation files for the current fold
        print(f"Fold {fold_idx + 1} Validation Files:")
        for i in val_index:
            print(f"File: {train_files[i]}, Label: {labels[i]}")

        # Load train data from both real and synthetic sources
        train_data = []
        for i in train_index:
            data = np.load(os.path.join(train_dirs[i], train_files[i]))
            train_data.append(data)
        train_data = np.vstack(train_data)
        train_labels = np.array([labels[i] for i in train_index])
        train_labels = np.repeat(train_labels, train_data.shape[0] // len(train_labels))
        train_data, train_labels = shuffle_data(train_data, train_labels)  # Shuffle train data

        # Load validation data from both real and synthetic sources
        val_data = []
        for i in val_index:
            data = np.load(os.path.join(train_dirs[i], train_files[i]))
            val_data.append(data)
        val_data = np.vstack(val_data)
        val_labels = np.array([labels[i] for i in val_index])
        val_labels = np.repeat(val_labels, val_data.shape[0] // len(val_labels))
        val_data, val_labels = shuffle_data(val_data, val_labels)

        # Save the data to .npy files
        np.save(os.path.join(folds_dir, f'X_train_{id}_fold_{fold_idx+1}.npy'), train_data)
        np.save(os.path.join(folds_dir, f'y_train_{id}_fold_{fold_idx+1}.npy'), train_labels)
        np.save(os.path.join(folds_dir, f'X_val_{id}_fold_{fold_idx+1}.npy'), val_data)
        np.save(os.path.join(folds_dir, f'y_val_{id}_fold_{fold_idx+1}.npy'), val_labels)
        np.save(os.path.join(folds_dir, f'X_test_{id}_fold_{fold_idx+1}.npy'), test_data)
        np.save(os.path.join(folds_dir, f'y_test_{id}_fold_{fold_idx+1}.npy'), test_labels_fold)

        print(f"Fold {fold_idx + 1}: Train indices: {len(train_index)}, Val indices: {len(val_index)}, Test indices: {len(test_index)}")

    print("\nArchivos .npy guardados para cada fold en la carpeta de folds.")
    print(f"Set '{id}' creado con éxito mezclando datos reales y sintéticos.")

In [3]:
prepare_and_save_folds(
    train_dir='../data/sets/test_together_N',
    test_dir='../data/sets/test_together_N',
    folds_dir='../data/sets/folds_5/files_real_N',
    id='real_N'
)

Carpeta '../data/sets/folds_5/files_real_N' eliminada.
Carpeta '../data/sets/folds_5/files_real_N' creada.
True
Fold 1 Test Files:
File: AVPEPUDEA0014_X.npy, Label: 1
File: AVPEPUDEA0016_X.npy, Label: 1
File: AVPEPUDEA0017_X.npy, Label: 1
File: AVPEPUDEA0020_X.npy, Label: 1
File: AVPEPUDEA0025_X.npy, Label: 1
File: AVPEPUDEA0031_X.npy, Label: 1
File: AVPEPUDEA0037_X.npy, Label: 1
File: AVPEPUDEA0042_X.npy, Label: 1
File: AVPEPUDEA0045_X.npy, Label: 1
File: AVPEPUDEA0055_X.npy, Label: 1
File: AVPEPUDEAC0004_X.npy, Label: 0
File: AVPEPUDEAC0012_X.npy, Label: 0
File: AVPEPUDEAC0018_X.npy, Label: 0
File: AVPEPUDEAC0031_X.npy, Label: 0
File: AVPEPUDEAC0041_X.npy, Label: 0
File: AVPEPUDEAC0043_X.npy, Label: 0
File: AVPEPUDEAC0048_X.npy, Label: 0
File: AVPEPUDEAC0050_X.npy, Label: 0
File: AVPEPUDEAC0053_X.npy, Label: 0
File: AVPEPUDEAC0057_X.npy, Label: 0
Fold 1 Train Files:
File: AVPEPUDEAC0034_X.npy, Label: 0
File: AVPEPUDEAC0001_X.npy, Label: 0
File: AVPEPUDEAC0045_X.npy, Label: 0
File: AV

In [4]:
prepare_and_save_folds(
    train_dir='../data/sets/generated_together_train_40_1e5_N',
    test_dir='../data/sets/test_together_N',
    folds_dir='../data/sets/folds_5/files_real_40_1e5_N',
    id='40_1e5_N'
)

Carpeta '../data/sets/folds_5/files_real_40_1e5_N' eliminada.
Carpeta '../data/sets/folds_5/files_real_40_1e5_N' creada.
True
Fold 1 Test Files:
File: AVPEPUDEA0014_X.npy, Label: 1
File: AVPEPUDEA0016_X.npy, Label: 1
File: AVPEPUDEA0017_X.npy, Label: 1
File: AVPEPUDEA0020_X.npy, Label: 1
File: AVPEPUDEA0025_X.npy, Label: 1
File: AVPEPUDEA0031_X.npy, Label: 1
File: AVPEPUDEA0037_X.npy, Label: 1
File: AVPEPUDEA0042_X.npy, Label: 1
File: AVPEPUDEA0045_X.npy, Label: 1
File: AVPEPUDEA0055_X.npy, Label: 1
File: AVPEPUDEAC0004_X.npy, Label: 0
File: AVPEPUDEAC0012_X.npy, Label: 0
File: AVPEPUDEAC0018_X.npy, Label: 0
File: AVPEPUDEAC0031_X.npy, Label: 0
File: AVPEPUDEAC0041_X.npy, Label: 0
File: AVPEPUDEAC0043_X.npy, Label: 0
File: AVPEPUDEAC0048_X.npy, Label: 0
File: AVPEPUDEAC0050_X.npy, Label: 0
File: AVPEPUDEAC0053_X.npy, Label: 0
File: AVPEPUDEAC0057_X.npy, Label: 0
Fold 1 Train Files:
File: AVPEPUDEAC0034_X.npy, Label: 0
File: AVPEPUDEAC0001_X.npy, Label: 0
File: AVPEPUDEAC0045_X.npy, Lab

In [5]:
prepare_and_save_folds(
    train_dir='../data/sets/generated_together_train_40_1e5_N',
    test_dir='../data/sets/generated_together_train_40_1e5_N',
    folds_dir='../data/sets/folds_5/files_syn_40_1e5_N',
    id='40_1e5_N'
)

Carpeta '../data/sets/folds_5/files_syn_40_1e5_N' eliminada.
Carpeta '../data/sets/folds_5/files_syn_40_1e5_N' creada.
True
Fold 1 Test Files:
File: AVPEPUDEA0014_X.npy, Label: 1
File: AVPEPUDEA0016_X.npy, Label: 1
File: AVPEPUDEA0017_X.npy, Label: 1
File: AVPEPUDEA0020_X.npy, Label: 1
File: AVPEPUDEA0025_X.npy, Label: 1
File: AVPEPUDEA0031_X.npy, Label: 1
File: AVPEPUDEA0037_X.npy, Label: 1
File: AVPEPUDEA0042_X.npy, Label: 1
File: AVPEPUDEA0045_X.npy, Label: 1
File: AVPEPUDEA0055_X.npy, Label: 1
File: AVPEPUDEAC0004_X.npy, Label: 0
File: AVPEPUDEAC0012_X.npy, Label: 0
File: AVPEPUDEAC0018_X.npy, Label: 0
File: AVPEPUDEAC0031_X.npy, Label: 0
File: AVPEPUDEAC0041_X.npy, Label: 0
File: AVPEPUDEAC0043_X.npy, Label: 0
File: AVPEPUDEAC0048_X.npy, Label: 0
File: AVPEPUDEAC0050_X.npy, Label: 0
File: AVPEPUDEAC0053_X.npy, Label: 0
File: AVPEPUDEAC0057_X.npy, Label: 0
Fold 1 Train Files:
File: AVPEPUDEAC0034_X.npy, Label: 0
File: AVPEPUDEAC0001_X.npy, Label: 0
File: AVPEPUDEAC0045_X.npy, Label

In [6]:
prepare_and_save_folds(
    train_dir='../data/sets/generated_together_train_40_1e5_N',
    test_dir='../data/sets/test_together_N',
    folds_dir='../data/sets/folds_5/files_syn_1_N',
    id='40_1e5_N'
)

Carpeta '../data/sets/folds_5/files_syn_1_N' eliminada.
Carpeta '../data/sets/folds_5/files_syn_1_N' creada.
True
Fold 1 Test Files:
File: AVPEPUDEA0014_X.npy, Label: 1
File: AVPEPUDEA0016_X.npy, Label: 1
File: AVPEPUDEA0017_X.npy, Label: 1
File: AVPEPUDEA0020_X.npy, Label: 1
File: AVPEPUDEA0025_X.npy, Label: 1
File: AVPEPUDEA0031_X.npy, Label: 1
File: AVPEPUDEA0037_X.npy, Label: 1
File: AVPEPUDEA0042_X.npy, Label: 1
File: AVPEPUDEA0045_X.npy, Label: 1
File: AVPEPUDEA0055_X.npy, Label: 1
File: AVPEPUDEAC0004_X.npy, Label: 0
File: AVPEPUDEAC0012_X.npy, Label: 0
File: AVPEPUDEAC0018_X.npy, Label: 0
File: AVPEPUDEAC0031_X.npy, Label: 0
File: AVPEPUDEAC0041_X.npy, Label: 0
File: AVPEPUDEAC0043_X.npy, Label: 0
File: AVPEPUDEAC0048_X.npy, Label: 0
File: AVPEPUDEAC0050_X.npy, Label: 0
File: AVPEPUDEAC0053_X.npy, Label: 0
File: AVPEPUDEAC0057_X.npy, Label: 0
Fold 1 Train Files:
File: AVPEPUDEAC0034_X.npy, Label: 0
File: AVPEPUDEAC0001_X.npy, Label: 0
File: AVPEPUDEAC0045_X.npy, Label: 0
File: 

In [7]:
prepare_and_save_folds(
    train_dir='../data/sets/test_together_N',
    test_dir='../data/sets/test_together_N',
    folds_dir='../data/sets/folds_5/files_syn_all_N',
    id='40_1e5_N'
)

Carpeta '../data/sets/folds_5/files_syn_all_N' eliminada.
Carpeta '../data/sets/folds_5/files_syn_all_N' creada.
True
Fold 1 Test Files:
File: AVPEPUDEA0014_X.npy, Label: 1
File: AVPEPUDEA0016_X.npy, Label: 1
File: AVPEPUDEA0017_X.npy, Label: 1
File: AVPEPUDEA0020_X.npy, Label: 1
File: AVPEPUDEA0025_X.npy, Label: 1
File: AVPEPUDEA0031_X.npy, Label: 1
File: AVPEPUDEA0037_X.npy, Label: 1
File: AVPEPUDEA0042_X.npy, Label: 1
File: AVPEPUDEA0045_X.npy, Label: 1
File: AVPEPUDEA0055_X.npy, Label: 1
File: AVPEPUDEAC0004_X.npy, Label: 0
File: AVPEPUDEAC0012_X.npy, Label: 0
File: AVPEPUDEAC0018_X.npy, Label: 0
File: AVPEPUDEAC0031_X.npy, Label: 0
File: AVPEPUDEAC0041_X.npy, Label: 0
File: AVPEPUDEAC0043_X.npy, Label: 0
File: AVPEPUDEAC0048_X.npy, Label: 0
File: AVPEPUDEAC0050_X.npy, Label: 0
File: AVPEPUDEAC0053_X.npy, Label: 0
File: AVPEPUDEAC0057_X.npy, Label: 0
Fold 1 Train Files:
File: AVPEPUDEAC0034_X.npy, Label: 0
File: AVPEPUDEAC0001_X.npy, Label: 0
File: AVPEPUDEAC0045_X.npy, Label: 0
Fi

In [8]:
prepare_and_save_folds(
    train_dir='../data/sets/generated_together_train_40_1e5_N',
    test_dir='../data/sets/test_together_syn_1_N',
    folds_dir='../data/sets/folds_5/files_syn_1_N',
    id='40_1e5_N'
)

Carpeta '../data/sets/folds_5/files_syn_1_N' eliminada.
Carpeta '../data/sets/folds_5/files_syn_1_N' creada.
True
Fold 1 Test Files:
File: AVPEPUDEA0014_X.npy, Label: 1
File: AVPEPUDEA0016_X.npy, Label: 1
File: AVPEPUDEA0017_X.npy, Label: 1
File: AVPEPUDEA0020_X.npy, Label: 1
File: AVPEPUDEA0025_X.npy, Label: 1
File: AVPEPUDEA0031_X.npy, Label: 1
File: AVPEPUDEA0037_X.npy, Label: 1
File: AVPEPUDEA0042_X.npy, Label: 1
File: AVPEPUDEA0045_X.npy, Label: 1
File: AVPEPUDEA0055_X.npy, Label: 1
File: AVPEPUDEAC0004_X.npy, Label: 0
File: AVPEPUDEAC0012_X.npy, Label: 0
File: AVPEPUDEAC0018_X.npy, Label: 0
File: AVPEPUDEAC0031_X.npy, Label: 0
File: AVPEPUDEAC0041_X.npy, Label: 0
File: AVPEPUDEAC0043_X.npy, Label: 0
File: AVPEPUDEAC0048_X.npy, Label: 0
File: AVPEPUDEAC0050_X.npy, Label: 0
File: AVPEPUDEAC0053_X.npy, Label: 0
File: AVPEPUDEAC0057_X.npy, Label: 0
Fold 1 Train Files:
File: AVPEPUDEAC0034_X.npy, Label: 0
File: AVPEPUDEAC0001_X.npy, Label: 0
File: AVPEPUDEAC0045_X.npy, Label: 0
File: 

In [9]:
a = np.load('../data/sets/folds_5/files_real_40_1e5_N/X_test_40_1e5_N_fold_2.npy')
b = np.load('../data/sets/folds_5/files_syn_40_1e5_N/X_test_40_1e5_N_fold_2.npy')
d = np.load('../data/sets/folds_5/files_syn_1_N/X_test_40_1e5_N_fold_2.npy')
e = np.load('../data/sets/folds_5/files_real_N/X_test_real_N_fold_2.npy')


## Crear set mezclado (files_all_real_syn_n)

Este set mezcla datos **reales y sintéticos** tanto para **entrenamiento** como para **evaluación**.

- **Train**: Datos reales + sintéticos
- **Validation**: Datos reales + sintéticos  
- **Test**: Datos reales + sintéticos
- **Folds**: 5 folds para cross-validation
- **Estratificación**: Se mantiene la proporción de clases (control/patológico)

In [10]:
# Crear el nuevo set files_all_real_syn_n que mezcla datos reales y sintéticos
# tanto para entrenamiento como para evaluación
prepare_and_save_folds_mixed(
    train_dir_real='../data/sets/test_together_N',  # Datos reales para entrenamiento
    train_dir_syn='../data/sets/generated_together_train_40_1e5_N',  # Datos sintéticos para entrenamiento
    test_dir_real='../data/sets/test_together_N',  # Datos reales para test
    test_dir_syn='../data/sets/generated_together_train_40_1e5_N',  # Datos sintéticos para test
    folds_dir='../data/sets/folds_5/files_all_real_syn_n',  # Directorio de salida
    id='all_real_syn_n'  # Identificador
)

Carpeta '../data/sets/folds_5/files_all_real_syn_n' creada.
Total archivos entrenamiento: 200 (Real: 100, Sintético: 100)
Total archivos test: 200 (Real: 100, Sintético: 100)

Fold 1 Test Files:
File: AVPEPUDEA0010_X.npy, Label: 1
File: AVPEPUDEA0011_X.npy, Label: 1
File: AVPEPUDEA0013_X.npy, Label: 1
File: AVPEPUDEA0020_X.npy, Label: 1
File: AVPEPUDEA0023_X.npy, Label: 1
File: AVPEPUDEA0031_X.npy, Label: 1
File: AVPEPUDEA0035_X.npy, Label: 1
File: AVPEPUDEA0038_X.npy, Label: 1
File: AVPEPUDEA0045_X.npy, Label: 1
File: AVPEPUDEA0049_X.npy, Label: 1
File: AVPEPUDEA0052_X.npy, Label: 1
File: AVPEPUDEA0055_X.npy, Label: 1
File: AVPEPUDEA0059_X.npy, Label: 1
File: AVPEPUDEAC0006_X.npy, Label: 0
File: AVPEPUDEAC0012_X.npy, Label: 0
File: AVPEPUDEAC0021_X.npy, Label: 0
File: AVPEPUDEAC0028_X.npy, Label: 0
File: AVPEPUDEAC0029_X.npy, Label: 0
File: AVPEPUDEAC0047_X.npy, Label: 0
File: AVPEPUDEAC0049_X.npy, Label: 0
File: AVPEPUDEAC0054_X.npy, Label: 0
File: AVPEPUDEA0002_X.npy, Label: 1
File:

In [11]:
# Verificar el nuevo set creado
# Cargar y mostrar información de un fold de ejemplo
fold_example = 1
X_train = np.load(f'../data/sets/folds_5/files_all_real_syn_n/X_train_all_real_syn_n_fold_{fold_example}.npy')
y_train = np.load(f'../data/sets/folds_5/files_all_real_syn_n/y_train_all_real_syn_n_fold_{fold_example}.npy')
X_val = np.load(f'../data/sets/folds_5/files_all_real_syn_n/X_val_all_real_syn_n_fold_{fold_example}.npy')
y_val = np.load(f'../data/sets/folds_5/files_all_real_syn_n/y_val_all_real_syn_n_fold_{fold_example}.npy')
X_test = np.load(f'../data/sets/folds_5/files_all_real_syn_n/X_test_all_real_syn_n_fold_{fold_example}.npy')
y_test = np.load(f'../data/sets/folds_5/files_all_real_syn_n/y_test_all_real_syn_n_fold_{fold_example}.npy')

print(f"Fold {fold_example} - Set files_all_real_syn_n:")
print(f"Train: {X_train.shape}, Labels: {y_train.shape}, Clase 0: {np.sum(y_train == 0)}, Clase 1: {np.sum(y_train == 1)}")
print(f"Val: {X_val.shape}, Labels: {y_val.shape}, Clase 0: {np.sum(y_val == 0)}, Clase 1: {np.sum(y_val == 1)}")
print(f"Test: {X_test.shape}, Labels: {y_test.shape}, Clase 0: {np.sum(y_test == 0)}, Clase 1: {np.sum(y_test == 1)}")

Fold 1 - Set files_all_real_syn_n:
Train: (8680, 11520), Labels: (8680,), Clase 0: 4340, Clase 1: 4340
Val: (1220, 11520), Labels: (1220,), Clase 0: 610, Clase 1: 610
Test: (2320, 11520), Labels: (2320,), Clase 0: 1160, Clase 1: 1160
